In [12]:
import tensorflow as tf
tf.__version__
import keras
keras.__version__

'2.9.1'

# 使用keras內建的MNIST手寫字符數據集
## 其中訓練集有60,000張28x28像素灰度圖像， 測試集則有10,000張

In [14]:
import cv2
import numpy as np
from keras.datasets import mnist
from keras import utils

In [15]:
# 載入訓練集
(x_train, y_train), (x_test, y_test) = mnist.load_data() 

In [16]:
# 訓練集資料
x_train = x_train.reshape(x_train.shape[0],-1)  # 轉換資料形狀 二為轉換成一維
x_train = x_train.astype('float32')/255         # 轉換資料型別 由於是圖片最大的是255，所以全部除以255
y_train = y_train.astype(np.float32)

In [17]:
# 測試集資料
x_test = x_test.reshape(x_test.shape[0],-1)     # 轉換資料形狀
x_test = x_test.astype('float32')/255           # 轉換資料型別
y_test = y_test.astype(np.float32)

In [18]:
knn=cv2.ml.KNearest_create()                    # 建立 KNN 訓練方法
knn.setDefaultK(5)                              # 參數設定
knn.setIsClassifier(True)

## 訓練後會將模型儲存為 mnist_knn.xml，儲存後會使用測試集進行測試

In [19]:
print('training...')
knn.train(x_train, cv2.ml.ROW_SAMPLE, y_train)  # 開始訓練
knn.save('mnist_knn.xml')                       # 儲存訓練模型
print('ok')

training...
ok


In [20]:
print('testing...')
test_pre = knn.predict(x_test)                  # 讀取測試集並進行辨識
test_ret = test_pre[1]
test_ret = test_ret.reshape(-1,)
print(test_ret)
print(y_test)
test_sum = (test_ret == y_test)   # test_lables
acc = test_sum.mean()                           # 得到準確率
print(acc)

testing...
[7. 2. 1. ... 4. 5. 6.]
[7. 2. 1. ... 4. 5. 6.]
0.9688


## 訓練好後就可以開始進行辨識，會先取出一個正方形的區域，將這個區域的像素轉成黑白的，轉換後在將尺寸縮小到 28x28 進行辨識，也會將辨識的影像顯示在原本影像的右上角。

In [21]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)                     # 啟用攝影鏡頭
print('loading...')
knn = cv2.ml.KNearest_load('mnist_knn.xml')   # 載入模型
print('start...')
if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    ret, img = cap.read()
    if not ret:
        print("Cannot receive frame")
        break
    img = cv2.resize(img,(540,300))          # 改變影像尺寸，加快處理效率
    x, y, w, h = 400, 200, 60, 60            # 定義擷取數字的區域位置和大小
    img_num = img.copy()                     # 複製一個影像作為辨識使用
    img_num = img_num[y:y+h, x:x+w]          # 擷取辨識的區域

    img_num = cv2.cvtColor(img_num, cv2.COLOR_BGR2GRAY)    # 顏色轉成灰階
    # 針對白色文字，做二值化黑白轉換，轉成黑底白字
    ret, img_num = cv2.threshold(img_num, 127, 255, cv2.THRESH_BINARY_INV)
    output = cv2.cvtColor(img_num, cv2.COLOR_GRAY2BGR)     # 顏色轉成彩色
    img[0:60, 480:540] = output                            # 將轉換後的影像顯示在畫面右上角

    img_num = cv2.resize(img_num,(28,28))   # 縮小成 28x28，和訓練模型對照
    img_num = img_num.astype(np.float32)    # 轉換格式
    img_num = img_num.reshape(-1,)          # 打散成一維陣列資料，轉換成辨識使用的格式
    img_num = img_num.reshape(1,-1)
    img_num = img_num/255
    img_pre = knn.predict(img_num)          # 進行辨識
    num = str(int(img_pre[1][0][0]))        # 取得辨識結果

    text = num                              # 印出的文字內容
    org = (x,y-20)                          # 印出的文字位置
    fontFace = cv2.FONT_HERSHEY_SIMPLEX     # 印出的文字字體
    fontScale = 2                           # 印出的文字大小
    color = (0,0,255)                       # 印出的文字顏色
    thickness = 2                           # 印出的文字邊框粗細
    lineType = cv2.LINE_AA                  # 印出的文字邊框樣式
    cv2.putText(img, text, org, fontFace, fontScale, color, thickness, lineType) # 印出文字

    cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)  # 標記辨識的區域
    cv2.imshow('lookhere', img)
    if cv2.waitKey(50) == ord('q'):
        break     # 按 q 鍵停止
cap.release()
cv2.destroyAllWindows()

loading...
start...
